# Computing Trajectory Metrics

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas/main?filepath=tutorials/6-trajectory-metrics.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/anitagraser/movingpandas-examples.


In [ ]:
import urllib
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from datetime import datetime, timedelta
from holoviews import opts, dim

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

## Raw AIS trajectories

In [ ]:
gdf = gpd.read_file('data/demodata_ais.gpkg')
gdf['t'] = pd.to_datetime(gdf['Timestamp'], format='%d/%m/%Y %H:%M:%S')
gdf = gdf[gdf.SOG>0]
gdf

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'MMSI', min_length=100, t='t')
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=30))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(traj_collection)))

In [ ]:
traj_gdf = trips.to_traj_gdf(agg={"ShipType": "mode", "MMSI": "mode", "SOG": ["min", "max"]})
traj_gdf